In [1]:
#!/cvmfs/soft.computecanada.ca/easybuild/software/2017/Core/python/3.5.4/bin/python

import netCDF4 as nc4
import numpy as np
from simpledbf import Dbf5
import time
import pandas as pd
import os



In [4]:
def nc_creation(Ntopo, Ntopo_flags):
    
    # remove sxisting files
    os.remove('./network_topology/network_topology.nc')

    # open the nc file to write
    ncid = nc4.Dataset('./network_topology/network_topology.nc', "w", format="NETCDF4")

    # the dimension of the nc file variables is equal to the row of the shapefile
    dimid_N = ncid.createDimension('n',Ntopo.shape[0])
    dimid_Nchar = ncid.createDimension('nchar',4)

    # going through the field one by one and saving them in nc varibales with long name, units and variable type (int, etc)
    for column in list(Ntopo_flags.columns):
        
        print(column, Ntopo_flags[column].iloc[0], Ntopo_flags[column].iloc[1], Ntopo_flags[column].iloc[2])
        
        
        
        # define the variable
        varid = ncid.createVariable(column, Ntopo_flags[column].iloc[0],('n',),\
                                    zlib=True,fill_value=Ntopo_flags[column].iloc[2]) #assuming all the fields are floats
        # Attributes
        varid.long_name      = column
        varid.unit           = Ntopo_flags[column].iloc[1]

        # Write data
        varid[:] = Ntopo[column]
    
    
    # define the variable PFAF
    varid = ncid.createVariable('PFAF','S1',('n','nchar',)) #assuming all the fields are floats
    # Attributes
    varid.long_name      = 'PFAF Order'
    varid.unit           = '-'
    # preparing the data
    data = np.full(Ntopo.shape[0], 'test', 'S4')
    data = nc4.stringtochar(data)
    varid[:,:]=data

    ncid.Conventions = 'CF-1.6'
    ncid.License     = 'The data were written by Shervan Gharari. They are under GPL.'
    ncid.history     = 'Created ' + time.ctime(time.time())
    ncid.source      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes).'
    ncid.close()




In [5]:
# load the date frame and create the netwrok topology
Ntopo = pd.read_csv('network_topology/NetTopo.csv')
Ntopo_flags = pd.read_csv('network_topology/NetTopo_flags.csv')

# 
nc_creation(Ntopo, Ntopo_flags)

ncid = nc4.Dataset('./network_topology/network_topology.nc')




is_lake int - -1
lake_model int - -1
basin_area f8 m**2 -9999
length f8 m -9999
slope f8 - -9999
lon f8 - -9999
lat f8 - -9999
hruid int - -1
seg_id int - -1
seg_hr_id int - -1
tosegment int - -1
width f8 m -9999
manning f8 - -9999
H06_irrigation int - -1
H06_Smax f8 m**3 -9999
H06_alpha f8 - -9999
H06_envfact f8 - -9999
H06_S_ini f8 m**3 -9999
H06_C1 f8 - -9999
H06_C2 f8 - -9999
H06_power f8 - -9999
H06_dinominator f8 - -9999
H06_c_compare f8 - -9999
H06_dead_S_Per f8 - -9999
H06_I_Mem_F int - -1
H06_I_Mem_L int - -1
H06_D_Mem_F int - -1
H06_D_Mem_L int - -1
H06_I_Jan f8 m**3 s**-1 -9999
H06_I_Feb f8 m**3 s**-2 -9999
H06_I_Mar f8 m**3 s**-3 -9999
H06_I_Apr f8 m**3 s**-4 -9999
H06_I_May f8 m**3 s**-5 -9999
H06_I_Jun f8 m**3 s**-6 -9999
H06_I_Jul f8 m**3 s**-7 -9999
H06_I_Aug f8 m**3 s**-8 -9999
H06_I_Sep f8 m**3 s**-9 -9999
H06_I_Oct f8 m**3 s**-10 -9999
H06_I_Nov f8 m**3 s**-11 -9999
H06_I_Dec f8 m**3 s**-12 -9999
H06_D_Jan f8 m**3 s**-13 -9999
H06_D_Feb f8 m**3 s**-14 -9999
H06_D_Mar

In [6]:
# create the input file for mizuRoute for a network topology of size 3

n_dim_length = 3
n_dim_time   = 10000

time_data = np.arange(n_dim_time)


if os.path.isfile('./network_topology/input.nc'):
    os.remove('./network_topology/input.nc')

with nc4.Dataset('./network_topology/input.nc', "w", format="NETCDF4") as ncid:

    dimid_N = ncid.createDimension('n', n_dim_length)  # only write one variable
    dimid_T = ncid.createDimension('time', n_dim_time)

    # Variables
    time_varid = ncid.createVariable('time', 'i4', ('time', ))
    # Attributes
    time_varid.long_name = 'time'
    time_varid.units = 'days since 1980-01-01 00:00:00'  # e.g. 'days since 1900-01-01 00:00'
    time_varid.calendar = 'gregorian'
    time_varid.standard_name = 'time'
    time_varid.axis = 'T'
    # Write data
    time_varid[:] = time_data

    # Variables
    ID_varid = ncid.createVariable('ID', 'int', ('n', ))
    # Attributes
    ID_varid.long_name = 'ID'
    ID_varid.units = '1'
    # Write data
    ID_varid[:] = np.array([1,2,3])

    # Variable
    # data_varid = ncid.createVariable(variable_name, 'f8', ('n','time', ), fill_value=-9999)
    data_varid = ncid.createVariable('runoff', 'f8', ('time','n', ), fill_value=-9999)
    # Attributes
    data_varid.long_name = 'runoff'
    data_varid.units = 'mm day**-1'
    # Write data
    data_varid[:] = np.random.random((n_dim_time,n_dim_length))*3
    
    # Variable
    # data_varid = ncid.createVariable(variable_name, 'f8', ('n','time', ), fill_value=-9999)
    data_varid = ncid.createVariable('evapo', 'f8', ('time','n', ), fill_value=-9999)
    # Attributes
    data_varid.long_name = 'Evaporation'
    data_varid.units = 'mm day**-1'
    # Write data
    data_varid[:] = 0
    
    # Variable
    # data_varid = ncid.createVariable(variable_name, 'f8', ('n','time', ), fill_value=-9999)
    data_varid = ncid.createVariable('precip', 'f8', ('time','n', ), fill_value=-9999)
    # Attributes
    data_varid.long_name = 'preipitation'
    data_varid.units = 'mm day**-1'
    # Write data
    data_varid[:] = 0

    ##
    ncid.Conventions = 'CF-1.6'
    ncid.License = 'The data were written by Shervan Gharari. Under Apache2.'
    ncid.history = 'Created ' + time.ctime(time.time())
    ncid.source = 'Written by script from library of Shervan Gharari (https://github.com/ShervanGharari/candex).'

